In [ ]:
# Imports
import logging

import matplotlib.pyplot as plt
import rasterio
from rasterio.windows import from_bounds
import pathlib
from pathlib import Path
from PIL import Image
import numpy as np
from pyproj import Transformer


In [ ]:
wgs84_to_utm32 = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)


def validate_input(latitude: float, longitude: float, radius: float = 100) -> None:
    logging.info(
        f"Incoming latitude '{latitude}' and longitude '{longitude}' with set radius '{radius}'."
    )

    if not -90 < latitude < 90 and not -180 < longitude < 180:
        error_message = (
            "Programm works with `EPSG:25832`. Looks like provided input data is in"
            "Geographic CRS (`EPSG:4326`). If it's not the case, ignore the warning!"
        )
        log.error(error_message)
        raise ValueError(error_message)

    if radius < 0 or radius > 100:
        log.warning(
            f"Radius expected to be from 0 meters till 100 meters. Received {radius} meters."
        )


def get_paths_of_required_images(
    latitude: int, longitude: int, radius: float = 100
) -> pathlib.Path:
    """Return paths of required images, that have part of final image."""
    ...


def get_image(latitude: float, longitude: float, radius: float = 100) -> Image.Image:
    """Generate 256x256 pixels Image on input latitude, longitude with selected radious.

    - 'latitude' and 'longitude' must be in 'EPSG:4326' EPSG:4326.
    - 'radius' value expected to be '0' till '100' meters.

    """
    validate_input(latitude=latitude, longitude=longitude, radius=radius)

    log.info(
        "Converting latitude and longitude from 'EPSG:4326' to 'EPSG:25832' format."
    )
    x_meters, y_meters = wgs84_to_utm32.transform(latitude, longitude)
    log.info(f"x_meters: {x_meters}, y_meters: {y_meters}")

    # Final image edge coordinates
    left = x_meters - radius
    bottom = y_meters + radius
    right = x_meters + radius
    top = y_meters - radius

    with rasterio.open(image_path) as src:
        window = from_bounds(left, bottom, right, top, transform=src.transform)

        # Prepare image to show
        partial_rgb = src.read([1, 2, 3], window=window)
        partial_rgb = np.transpose(partial_rgb, (1, 2, 0))
        partial_rgb = np.clip(partial_rgb, 0, 255).astype(np.uint8)
        pil_img = Image.fromarray(partial_rgb, mode="RGB")
        pil_img_256 = pil_img.resize((256, 256), resample=Image.BILINEAR)

        # Plot the data
        plt.figure(figsize=(5, 5))
        plt.imshow(pil_img_256)
        plt.title("256x256 Geospatial Data Visualization")
        plt.axis("off")
        plt.show()

In [ ]:
get_image(51.962291, 7.626426, 100)

[2025-01-12 15:08:27,914] [INFO] Incoming latitude '51.962291' and longitude '7.626426' with set radius '100'.
[2025-01-12 15:08:27,915] [INFO] Converting latitude and longitude from 'EPSG:4326' to 'EPSG:25832' format.
[2025-01-12 15:08:27,916] [INFO] x_meters: 5737501.056387628, y_meters: 1149370.1175921818


WindowError: Bounds and transform are inconsistent